### paketler ve data

In [1]:
import requests
import pandas as pd
import sqlite3
import tweepy
import twitter_config

client = tweepy.Client(bearer_token=twitter_config.BEARER_TOKEN, wait_on_rate_limit=True)

### github listesi

In [2]:
#get data from link
url = 'https://theunitedstates.io/congress-legislators/legislators-current.csv'
github = pd.read_csv(url, dtype={'twitter_id': str})

In [3]:
github = github[github["type"] == "sen"]
github

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,...,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,...,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,...,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,...,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,Britt,Katie,Elizabeth,NaN,NaN,Katie Boyd Britt,1982-02-02,F,sen,AL,...,NaN,S416,S2AL00145,NaN,456874,NaN,Katie Britt,NaN,NaN,Katie Britt
462,Schmitt,Eric,Stephen,NaN,NaN,Eric Schmitt,1975-06-20,M,sen,MO,...,NaN,S420,S2MO00544,NaN,456875,NaN,Eric Schmitt,NaN,NaN,Eric S. Schmitt
463,Vance,J.D. (James),(David),NaN,NaN,J.D. Vance,1984-08-02,M,sen,OH,...,NaN,S421,S2OH00436,NaN,456876,NaN,J.D. Vance,NaN,NaN,J. D. Vance
464,Fetterman,John,Karl,NaN,NaN,John Fetterman,1969-08-15,M,sen,PA,...,NaN,S418,S6PA00274,NaN,456877,166286.0,John Fetterman,NaN,NaN,John Fetterman


In [5]:
github.columns

Index(['last_name', 'first_name', 'middle_name', 'suffix', 'nickname',
       'full_name', 'birthday', 'gender', 'type', 'state', 'district',
       'senate_class', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'twitter_id', 'facebook', 'youtube', 'youtube_id',
       'mastodon', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')

In [6]:
dbconn = sqlite3.connect('twitter003.db')
cursor = dbconn.cursor()

In [25]:
# create sqlite table using columns from github dataframe
cursor.execute('''CREATE TABLE github_list(
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              last_name TEXT,
              first_name TEXT,
              middle_name TEXT,
              suffix TEXT,
              nickname TEXT,
              full_name TEXT,
              birthday TEXT,
              gender TEXT,
              type TEXT,
              state TEXT,
              district TEXT,
              senate_class TEXT,
              party TEXT,
              url TEXT,
              address TEXT,
              phone TEXT,
              contact_form TEXT,
              rss_url TEXT,
              twitter TEXT,
              twitter_id TEXT,
              facebook TEXT,
              youtube TEXT,
              youtube_id TEXT,
              mastodon TEXT,
              bioguide_id TEXT,
              thomas_id TEXT,
              opensecrets_id TEXT,
              lis_id TEXT,
              fec_ids TEXT,
              cspan_id TEXT,
              govtrack_id TEXT,
              votesmart_id TEXT,
              ballotpedia_id TEXT,
              washington_post_id TEXT,
              icpsr_id TEXT,
              wikipedia_id TEXT
              )''')

In [26]:
# github list to sqlite
github.to_sql('github_list', dbconn, if_exists='append', index = False)

100

In [27]:
# create sqlite tweets table using columns from above
cursor.execute('''CREATE TABLE tweets(
              id_p INTEGER PRIMARY KEY AUTOINCREMENT,
              author_id TEXT,
              text TEXT,
              context_annotations TEXT,
              conversation_id TEXT,
              created_at TEXT,
              edit_controls TEXT,
              entities TEXT,
              geo TEXT,
              id TEXT UNIQUE,
              in_reply_to_user_id TEXT,
              lang TEXT,
              public_metrics TEXT,
              possibly_sensitive TEXT,
              referenced_tweets TEXT,
              reply_settings TEXT,
              source TEXT,
              withheld TEXT,
              attachments TEXT,
              errors TEXT
              )''')

In [7]:
# create a dataframe named tweetsTable
tweetsTable = pd.DataFrame(columns=['author_id','text','context_annotations','conversation_id','created_at','edit_controls','entities','geo','id','in_reply_to_user_id','lang','public_metrics','possibly_sensitive','referenced_tweets','reply_settings','source','withheld','attachments','errors'])
tweetsTable["possibly_sensitive"] = tweetsTable["possibly_sensitive"].astype(bool)
tweetsTable

,author_id,text,context_annotations,conversation_id,created_at,edit_controls,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,withheld,attachments,errors


In [8]:
github.columns

Index(['last_name', 'first_name', 'middle_name', 'suffix', 'nickname',
       'full_name', 'birthday', 'gender', 'type', 'state', 'district',
       'senate_class', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'twitter_id', 'facebook', 'youtube', 'youtube_id',
       'mastodon', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')

In [9]:
colums_toget = ['twitter_id']
for i in colums_toget:
  print(i)

twitter_id


In [10]:
a = 1
b = 0
for j in colums_toget:
  for i in github[j]:
    if str(i) == "nan":
      pass
    else:
      print(i)
      b += 1
      if int(b) % 500 == 0:
        a += 1
        filename = "tweetsTable" + str(a) + ".csv"
        tweetsTable.to_csv(filename)
        tweetsTable = tweetsTable.astype(str)
        tweetsTable.to_sql('tweets', dbconn, if_exists='append', index=False)
        # frop all rows from tweetsTable
        tweetsTable.drop(tweetsTable.index, inplace=True)
      if i not in github["twitter_id"]:
        if i is not None:
          try:
            for response in tweepy.Paginator(client.get_users_tweets,
                                            id=i,
                                            start_time='2022-10-01T00:00:00Z', end_time='2022-12-10T00:00:00Z',
                                            expansions = ['attachments.poll_ids,attachments.media_keys,author_id,edit_history_tweet_ids,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'],
                                            tweet_fields = ['attachments,author_id,context_annotations,conversation_id,created_at,edit_controls,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'], max_results=100):
              if response.data is None:
                pass
              if response.data is not None:                            
                for i in response.data:
                  tweetsTable = pd.concat([tweetsTable, pd.DataFrame([{'author_id':i.author_id,
                                  'text':i.text,
                                  'context_annotations':i.context_annotations,
                                  'conversation_id':i.conversation_id,
                                  'created_at':i.created_at,
                                  'edit_controls':i.edit_controls,
                                  'entities':i.entities,
                                  'geo':i.geo, 'id':i.id,
                                  'in_reply_to_user_id':i.in_reply_to_user_id,
                                  'lang':i.lang,
                                  'public_metrics':i.public_metrics,
                                  'possibly_sensitive':i.possibly_sensitive,
                                  'referenced_tweets':i.referenced_tweets,
                                  'reply_settings':i.reply_settings,
                                  'source':i.source,
                                  'withheld':i.withheld,
                                  'attachments':i.attachments,
                                  'errors':response.errors}])], ignore_index=True)
          except:
            print(i)
            print("bunda hata var")

43910797
117501995
109071031
249787913
171598736
476256944
22044727
18695134
29442313
76456274
515822213
242555999
202206694
264219447
19726613
13218102
247334603
14845376
339822881
29201047
486694111
1096059529
109287731
7429102
899978622416695297
15324851
234374703
1074518754
224285242
278145569
278124059
5558312
193794406
1017500185356853248
600463589
10615232
1099199839
92186819
382791093
233737858
225921757
88784440
19318314
21406834
18632666
18061669
2853793517
293131808
1298624375692894210
236511574
229966028
17494010
217543151
296361085
18137749
1410590874
250188760
234128524
1262099252
968650362
1080844782
1058520120
970207298
1068481578
11651202
1048784496
1071402577
1060370282
1074480192
2167097881
2891210047
2856787757
2964174789
2955485182
816683274076614656
946549322
811313565760163844
1336344005588738052
817138492614524928
818554054309715969
941000686275387392
983348251972816896
131546062
1080870981877534720
1080960924687704064
1078693601356509184
1324926274888888320
228

In [22]:
tweetsTable

,author_id,text,context_annotations,conversation_id,created_at,edit_controls,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,withheld,attachments,errors


In [11]:
tweetsTable = tweetsTable.astype(str)
tweetsTable.to_sql('tweets', dbconn, if_exists='append', index=False)

16166

### birleştirme

In [5]:
import pandas as pd
from pangres import upsert
from sqlalchemy import create_engine

In [101]:
# connect to sqlite using sqlalchemy
engine = create_engine('sqlite:///twitter003.db', echo=False)

In [91]:
# get all csv files to a list
import glob
csv_files = glob.glob('*.csv')

In [ ]:
for i in csv_files[5:-1]:
  #read csv and make index col as id
  print(i)
  df = pd.read_csv(i, index_col='id', dtype={'id': str})
  # drop duplicates at column id
  df = df[~df.index.duplicated(keep='first')]
  #if a dataframe has a column named Unnamed: 0, drop it
  if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
  print(len(df))
  iterator = upsert(con=engine, df=df, table_name='tweets', if_row_exists='ignore', chunksize=1000, create_table=False, yield_chunks=True)
  for result in iterator:
      print(f'{result.rowcount} row(s) updated')


In [100]:
# close the connection
engine.dispose()

### bağlantıyı kapat

In [50]:
#close connection
dbconn.close()